In [33]:
import torch
import torch.nn as nn
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

In [34]:
# device config for GPU support

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [35]:
# hyper parameters

n_epochs = 8
batch_size = 4
learning_rate = 0.0004

In [36]:
# transforms and datasets

# dataset has range of [0,1], we are transforming to [-1.1]
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

train = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(dataset=train, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(dataset=test, shuffle=False, batch_size=batch_size)
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse','ship', 'truck']

Files already downloaded and verified
Files already downloaded and verified


In [37]:
# model

class ConvNet(nn.Module):
    
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16*5*5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
model = ConvNet().to(device)

In [38]:
# loss and optimizer

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
n_total_steps = len(train_loader)

for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # original shape [4,3,32,32] = [4,3,1024]
        # input_later = 3 input channels (rgb), 6 output channels, 5 kernel size
        images = images.to(device)
        labels = labels.to(device)
        
        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 2000 == 0:
            print(f'epoch {epoch+1}/{n_epochs}, step {i+1}/{n_total_steps}, loss: {loss.item():.4f}')


epoch 1/8, step 2000/12500, loss: 2.3084
epoch 1/8, step 4000/12500, loss: 2.3608
epoch 1/8, step 6000/12500, loss: 2.2890
epoch 1/8, step 8000/12500, loss: 2.2690
epoch 1/8, step 10000/12500, loss: 2.3149
epoch 1/8, step 12000/12500, loss: 2.2685
epoch 2/8, step 2000/12500, loss: 2.2946
epoch 2/8, step 4000/12500, loss: 2.2971
epoch 2/8, step 6000/12500, loss: 2.2729
epoch 2/8, step 8000/12500, loss: 2.2829
epoch 2/8, step 10000/12500, loss: 2.2741
epoch 2/8, step 12000/12500, loss: 2.2079
epoch 3/8, step 2000/12500, loss: 1.9644
epoch 3/8, step 4000/12500, loss: 2.0525
epoch 3/8, step 6000/12500, loss: 2.6676
epoch 3/8, step 8000/12500, loss: 1.5187
epoch 3/8, step 10000/12500, loss: 1.4789
epoch 3/8, step 12000/12500, loss: 1.9248
epoch 4/8, step 2000/12500, loss: 1.3216
epoch 4/8, step 4000/12500, loss: 1.6703
epoch 4/8, step 6000/12500, loss: 1.5048
epoch 4/8, step 8000/12500, loss: 1.1851
epoch 4/8, step 10000/12500, loss: 2.1346
epoch 4/8, step 12000/12500, loss: 2.8445
epoch 5/

In [18]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    n_class_correct = [0 for i in range(10)]
    n_class_samples = [0 for i in range(10)]
    
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        
        _, predicted = torch.max(outputs, 1)
        n_samples += labels.size(0)
        n_correct += (predicted == labels).sum().item()
        
        for i in range(batch_size):
            label = labels[i]
            pred = predicted[i]
            if (label == pred):
                n_class_correct[label] += 1
            n_class_samples[label] += 1
            
    acc = (n_correct/n_samples) * 100
    print(f'Accuracy of network: {acc}')
    
    for i in range(10):
        acc = (n_class_correct[i]/n_class_samples[i]) * 100
        print(f'Accuracy of {classes[i]}: {acc}')
        

Accuracy of network: 58.53
Accuracy of plane: 61.1
Accuracy of car: 63.3
Accuracy of bird: 34.1
Accuracy of cat: 31.7
Accuracy of deer: 57.99999999999999
Accuracy of dog: 49.4
Accuracy of frog: 70.39999999999999
Accuracy of horse: 72.5
Accuracy of ship: 72.2
Accuracy of truck: 72.6
